In [7]:
%load_ext autoreload
%autoreload 2
from simuq import QSystem
from simuq import Qubit
from simuq.braket import BraketProvider
bp = BraketProvider()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from simuq.qutip import QuTiPProvider

qtpp = QuTiPProvider()


def tv(P1, P2):
    keys = set(P1.keys()).union(P2.keys())
    s = 0
    for k in keys:
        if k not in P1.keys():
            s += P2[k]
        elif k not in P2.keys():
            s += P1[k]
        else:
            s += abs(P1[k] - P2[k])
    return s / 2

In [13]:
circ_list = []
chain = True
n = 6
for T in [0.2, 0.4, 0.6, 0.8, 1]:
    qs = QSystem()
    qubits = [Qubit(qs) for i in range(n)]
    H0 = 0
    if chain:
        for i in range(n - 1):
            H0 += qubits[i].Z * qubits[(i + 1) % n].Z
    else:
        for i in range(n):
            H0 += qubits[i].Z * qubits[(i + 1) % n].Z

    for i in range(n):
        H0 += -qubits[i].X

    qs.add_evolution(H0, T)
    bp.compile(qs, provider="ionq", device="Harmony", aais="heisenberg", tol=1, trotter_num=4)
    bp.run(shots=1024, on_simulator=True)
    result = bp.results()
    qtpp.compile(qs)
    qtpp.run()
    result_ideal = qtpp.results()
    print(f"Distance to ideal state: {tv(result,result_ideal)}")

Submitted.
Compiled.
Solved.
Distance to ideal state: 0.02273218481305408
Submitted.
Compiled.
Solved.
Distance to ideal state: 0.07156694499294528
Submitted.
Compiled.
Solved.
Distance to ideal state: 0.08289717248959154
Submitted.
Compiled.
Solved.
Distance to ideal state: 0.09254468740216361
Submitted.
Compiled.
Solved.
Distance to ideal state: 0.13434968143865
